In [21]:
import sys
sys.path.append('..')

from personai.core.steps import (
    StartStep, 
    EndStep,
    SummarizationStep,
    TranslationStep,
    ControlStep,
    ApiCallStep,
    ValidationStep
)

from personai.core.models import OpenAIModel
import ast

In [22]:
openai_token = 'some_token'

# Summarize

In [23]:
openai_model_name = 'gpt-3.5-turbo-16k-0613'

model = OpenAIModel(openai_model_name, openai_token)

In [24]:
text = "Sending end user IDs in your requests can be a useful tool to help OpenAI track and detect abuse. This allows OpenAI to provide more actionable feedback to your team if we detect any policy violations in your application."

summarize_step = SummarizationStep(model)
print(summarize_step(text, "summarize", 10))

Sending end user IDs helps OpenAI track and detect abuse.


# Translate

In [5]:
openai_model_name = 'gpt-3.5-turbo-16k-0613'

model = OpenAIModel(openai_model_name, openai_token, temperature=0)

In [6]:
text = "Sending user [IDs] helps {OpenAI} track and detect abuse."
translate_step = TranslationStep(model)
translate_step(text, "english", "russian", "formal")

'Отправка идентификаторов пользователей помогает {OpenAI} отслеживать и обнаруживать злоупотребления.'

In [7]:
text = "Sending [user identifiers] helps {OpenAI} track and detect abuse."
print(translate_step(text, "english", "russian", "formal"))

Отправка [идентификаторов пользователей] помогает {OpenAI} отслеживать и обнаруживать злоупотребления.


# Validate

In [8]:
openai_model_name = 'gpt-3.5-turbo-16k-0613'

validation_model = OpenAIModel(openai_model_name, openai_token, temperature=0)
validation_step = ValidationStep(validation_model)

In [9]:
text="Sending end user IDs in your requests can be a useful tool to help OpenAI track and detect abuse. \
This allows OpenAI to provide more actionable feedback to your team if we detect any policy violations in your application."

task = "Translate following text from english to russian"
answer = "Отправка идентификаторов конечных пользователей в ваши запросы может быть полезным инструментом для помощи OpenAI \
    в отслеживании и обнаружении злоупотреблений. Это позволяет OpenAI предоставлять более действенные отзывы вашей команде, \
        если мы обнаружим нарушения политики в вашем приложении."

In [10]:
validation_step(answer=answer, task=task, context=text)

'{"evaluation": "valid", "explanation": ""}'

In [11]:
answer = "Привет, как дела?"
validation_step(answer=answer, task=task, context=text)

'{"evaluation": "invalid", "explanation": "The answer is not relevant to the context. The user\'s answer is a greeting and does not translate the given English text to Russian."}'

# Api Call

In [16]:
def search_call(**search_params):
    return "Search"

def info_call(**search_params):
    return "Info"


search_call_spec = [
        dict(
            name='search_call',
            description='search API call with the given parameters to make books recommendation',
            parameters=dict(
                type='object',
                properties=dict(
                    authorName=dict(type='string', description="full author name"),
                    bookName=dict(type='string', description="full book name"),
                    priceMin=dict(type='integer', description="minimum price in rubles"),
                    priceMax=dict(type='integer', description="maximum price in rubles"),
                    adultsOnly=dict(type='boolean', description="only books for adults", enum=["True", "False"]),
                    childOnly=dict(type='boolean', description="only child books", enum=["True", "False"]),
                    illustrations=dict(type='boolean', enum=["True", "False"])))),
        dict(
            name='info_call',
            description='info API call to get info about the selected book',
            parameters=dict(
                type='object',
                properties=dict(
                    bookName=dict(type='string', description="full book name")
                )))
]

In [17]:
openai_model_name = 'gpt-3.5-turbo-16k-0613'

api_call_model = OpenAIModel(openai_model_name, openai_token, temperature=0)
api_call_step = ApiCallStep(api_call_model, {"search_call": search_call, "info_call": info_call}, search_call_spec)

In [18]:
task="childern's book no more than 500 rubles with pictures about knights"
api_call_step(task)

{'result': 'Search',
 'function_call': '{\'name\': \'search_call\', \'arguments\': "{\'childOnly\': True, \'priceMax\': 500, \'illustrations\': True}"}'}

In [19]:
task="""what's the price of the "Shining" book."""
api_call_step(task)

{'result': 'Info',
 'function_call': '{\'name\': \'info_call\', \'arguments\': "{\'bookName\': \'Shining\'}"}'}

In [20]:
task="rock"
api_call_step(task)

{'result': None,
 'message': 'I\'m sorry, but I\'m not sure what you mean by "rock". Could you please provide more context or clarify your request?'}

# Control

In [15]:
openai_model_name = 'gpt-3.5-turbo-16k-0613'

summarize_model = OpenAIModel(openai_model_name, openai_token)
translate_model = OpenAIModel(openai_model_name, openai_token, temperature=0)

summarize_step = SummarizationStep(summarize_model)
translate_step = TranslationStep(translate_model)

start_step = StartStep(summarize_model)
end_step = EndStep(summarize_model)
steps = {step.name: step for step in [summarize_step, translate_step, end_step]}
steps_specs = [step.spec for step in steps.values()]

In [16]:
control_model = OpenAIModel(openai_model_name, openai_token, history_volume=5, temperature=0)
control_step = ControlStep(control_model, steps, steps_specs)

In [17]:
text = "Sending end user IDs in your requests can be a useful tool to help OpenAI track and detect abuse. \
This allows OpenAI to provide more actionable feedback to your team if we detect any policy violations in your application."
task = f"Highlight the main idea of the text using no more 10 words and translate it into Russian: {text}"

In [18]:
is_end = False
iteration = 0
print('- START')
while not is_end:
    iteration += 1
    response = control_step(task)['result']
    print(f'{iteration}. {response}')
    task = response
    func_call = control_model.history[-1].get('function_call')
    if func_call:
        func_call = ast.literal_eval(func_call)
        if func_call.get('name') == 'end_step':
            is_end = True
            print('- DONE')

- START
1. Sending user IDs to OpenAI helps track abuse and improve feedback.
2. Отправка идентификаторов пользователей в OpenAI помогает отслеживать злоупотребления и улучшать обратную связь.
3. Идентификаторы пользователей в OpenAI помогают отслеживать злоупотребления и улучшать обратную связь.
4. Идентификаторы пользователей в OpenAI для безопасности и обратной связи.
5. Идентификаторы пользователей в OpenAI используются для обеспечения безопасности и улучшения обратной связи.
- DONE


In [19]:
control_model.history

[{'role': 'user',
  'content': 'Sending user IDs to OpenAI helps track abuse and improve feedback.'},
 {'role': 'assistant',
  'content': 'Отправка идентификаторов пользователей в OpenAI помогает отслеживать злоупотребления и улучшать обратную связь.',
  'function_call': '{\'name\': \'translation_step\', \'arguments\': "{\'text\': \'Sending user IDs to OpenAI helps track abuse and improve feedback.\', \'from_language\': \'en\', \'to_language\': \'ru\', \'style\': \'default\'}"}'},
 {'role': 'user',
  'content': 'Отправка идентификаторов пользователей в OpenAI помогает отслеживать злоупотребления и улучшать обратную связь.'},
 {'role': 'assistant',
  'content': 'Идентификаторы пользователей в OpenAI помогают отслеживать злоупотребления и улучшать обратную связь.',
  'function_call': '{\'name\': \'summarization_step\', \'arguments\': "{\'text\': \'Отправка идентификаторов пользователей в OpenAI помогает отслеживать злоупотребления и улучшать обратную связь.\', \'action\': \'summarize\', 